# pCrunch's Crunch class

The `Crunch` class is a general analysis tool for batches of time-series based data across multiple environmental conditions (i.e., a full wind speed and turbulence seed sweep). The methods are agnostic to the aeroelastic multibody simulation tool (OpenFAST or HAWC2 or Bladed or QBlade or in-house equivalents). The `AeroelasticOutput` class provides the data containers for each individual simulation.  The `AeroelasticOutput` class provides many analysis capabilities and the `Crunch` class extends them into their batch versions.

The `Crunch` class supports keeping all time series data in memory and a lean "streaming" version where outputs are processed and then deleted, retaining only the critical statistics and analysis outputs.

This file lays out some workflows and showcases capabilities of the `Crunch` class.  It is probably best to walk through the examples of the `AeroelasticOutput` class first.

## Creating a new class instance

The `Crunch` class can be initialized from a list of AeroelasticOutput instances or none, in order to setup a "streaming" analysis.  Pleaes see the AeroelasticOutput example for the various means to initialize one of its instances.  pCrunch provides a reader for OpenFAST output files (both binary and ascii) and common Python data structures are also supported.  To extend pCrunch for use with other aeroelastic multibody codes, users could simply use the `openfast_readers.py` file as a template.  Here are some examples:

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pCrunch import Crunch, read, FatigueParams

thisdir = os.path.realpath('')
datadir = os.path.join(thisdir, '..', 'pCrunch', 'test', 'data')

# OpenFAST output files
filelist = glob.glob( os.path.join(datadir, '*.out') )
filelist.sort()
print(f"Found {len(filelist)} files.")

# Read all outputs into a list
outputs = [read(m) for m in filelist[1:]]

# Vector magnitudes
mc = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Channel-specific fatigue properties
fc = {
    "RootMc1": FatigueParams(lifetime=25.0, slope=10.0, ultimate_stress=6e8, load2stress=250.0, S_intercept=5e9),
    "RootMc2": FatigueParams(lifetime=25.0, slope=10.0, ultimate_stress=6e8, load2stress=250.0, S_intercept=5e9),
    "RootMc3": FatigueParams(lifetime=25.0, slope=10.0, ultimate_stress=6e8, load2stress=250.0, S_intercept=5e9),
}

# Channels to focus on for extreme event tabulation
ec = ["RotSpeed", "RotThrust", "RotTorq"]

# Standard use case with all outputs read prior to use of Crunch.
mycruncher = Crunch(outputs)

# Can also add some batch data operations in the constructor (many more available in Batch Processing below)
mycruncher_mc = Crunch(outputs, magnitude_channels=mc, trim_data=[40, 80], fatigue_channels=fc, extreme_channels=ec)

# When planning on adding outputs later, you still need create a Crunch object that is initially empty of data
# The `lean` flag says that the outputs should be processed, but not stored in memory
mycruncher_lean = Crunch(outputs=[], lean=True)

# Can still add the batch operations to be done later when outputs are added
mycruncher_lean_mc = Crunch(outputs=[], lean=True, magnitude_channels=mc, trim_data=[40,80], fatigue_channels=fc, extreme_channels=ec)

Found 4 files.


## Crunching the data

### With full memory storage
The Crunch class can batch process the outputs using one or more processors up to the number available workstation cores.  This computes the essential statistics for each output.

In [2]:
# Process all outputs in parallel
mycruncher.process_outputs(cores=1)

# Process all outputs and override any prior input setting (especially in fatigue calculation)
mycruncher_mc.process_outputs(return_damage=True)

The key outputs that are stacked together for each output are:

- Summary statistics
- Load ranking
- Extreme event table
- Damage equivalent loads (DELs)
- Palmgren-Miner damage

In [3]:
# The summary stats per each file are here:
mycruncher.summary_stats

,Time,WindVxi,WindVyi,WindVzi,WaveElev,Wave1Vxi,Wave1Vyi,Wave1Vzi,Wave1Axi,Wave1Ayi,...,Fair8Ang,Anch8Ten,Anch8Ang,TipSpdRat,RotCp,RotCt,RotCq,RootMc1,RootMc2,RootMc3
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
DLC2.3_1.out,40.0,8.200,0.0,0.0,-0.812,-0.5420,0.0,-0.7480,-0.7230,0.0,...,3289.97500,0.0,0.0,200.575785,7.928011,19.910354,1.038984,205721.341721,213361.952123,222926.620682
DLC2.3_2.out,40.0,8.197,0.0,0.0,-1.055,-0.5438,0.0,-0.5059,-0.3195,0.0,...,3284.85275,0.0,0.0,202.995350,8.196020,20.209087,1.063160,209783.673742,218196.946790,226368.344817
DLC2.3_3.out,40.0,8.197,0.0,0.0,-1.135,-0.5403,0.0,-0.5156,-0.2482,0.0,...,3286.98625,0.0,0.0,202.480952,8.109685,20.148358,1.055470,209258.712717,216929.128758,225974.727664


In [4]:
# These are indexable by channel, stat:
mycruncher.summary_stats["RootMc1"]

,min,max,std,mean,median,abs,integrated
DLC2.3_1.out,459.805830,9134.167593,2707.224813,5142.646328,6147.974788,9134.167593,205721.341721
DLC2.3_2.out,277.648587,9079.452302,2709.503118,5244.431821,6430.205456,9079.452302,209783.673742
DLC2.3_3.out,347.604352,8986.223847,2707.345218,5231.190499,6669.062227,8986.223847,209258.712717


In [5]:
mycruncher.summary_stats["RootMc1"]['min']

DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: min, dtype: float64

In [6]:
# Or by file
mycruncher.summary_stats.loc["DLC2.3_1.out"]

Time      min               40.000000
WindVxi   min                8.200000
WindVyi   min                0.000000
WindVzi   min                0.000000
WaveElev  min               -0.812000
                            ...      
RotCt     integrated        19.910354
RotCq     integrated         1.038984
RootMc1   integrated    205721.341721
RootMc2   integrated    213361.952123
RootMc3   integrated    222926.620682
Name: DLC2.3_1.out, Length: 952, dtype: float64

In [7]:
# Load rankings are manipulations of the summary statistics table
# All channels and statistics are available
mycruncher.get_load_rankings(['RootMc1'],['max'])

,file,channel,stat,val
0,DLC2.3_1.out,RootMc1,max,9134.167593


In [8]:
# Damage equivalent loads are found here:
mycruncher_mc.dels

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,2557.248362,5801.906159,2074.270412
DLC2.3_2.out,2759.655817,4632.573610,2138.262799
DLC2.3_3.out,2791.460474,5839.945621,2213.542813


In [9]:
# Palmgren-Miner damage can be viewed with (although it is not computed without a `return_damage=True`
mycruncher_mc.damage

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,NaN,NaN,NaN
DLC2.3_2.out,NaN,NaN,NaN
DLC2.3_3.out,NaN,NaN,NaN


In [10]:
# Extreme events table. For each channel, there is a list of the extreme condition for each output case
mycruncher.extremes

{'RotSpeed': [{'Time': 61.8,
   'RotSpeed': 11.1,
   'RotTorq': 844.0,
   'RotThrust': 369.0},
  {'Time': 61.9, 'RotSpeed': 11.28, 'RotTorq': 159.4, 'RotThrust': 367.0},
  {'Time': 61.9, 'RotSpeed': 11.33, 'RotTorq': 140.8, 'RotThrust': 317.0}],
 'RotTorq': [{'Time': 54.45,
   'RotSpeed': 10.6,
   'RotTorq': 2650.0,
   'RotThrust': 546.0},
  {'Time': 54.4, 'RotSpeed': 10.74, 'RotTorq': 2701.0, 'RotThrust': 554.0},
  {'Time': 54.4, 'RotSpeed': 10.61, 'RotTorq': 2638.0, 'RotThrust': 575.1}],
 'RotThrust': [{'Time': 51.6,
   'RotSpeed': 10.1,
   'RotTorq': 2410.0,
   'RotThrust': 759.0},
  {'Time': 60.35, 'RotSpeed': 10.41, 'RotTorq': -1041.0, 'RotThrust': 786.9},
  {'Time': 60.35, 'RotSpeed': 10.48, 'RotTorq': -1046.0, 'RotThrust': 746.2}]}

### Crunching in "lean / streaming" mode

If operating in "lean / streaming" mode, the outputs can either be processed one at a time, or even more lean, the summary statistics themselves can be passed to the `cruncher` object to append to the running list.

In [11]:
# Adding AeroelasticOutput objects in lean / streaming mode
for iout in outputs:
    mycruncher_lean.add_output( iout ) # Each output is processed without retaining the full time series

# Adding statistics incrementally
results_pool = []
for iout in outputs:
    iresults = mycruncher_lean_mc.process_single( iout ) # This could be the result of parallelized function
    results_pool.append( iresults )

# After parallel processing is complete, assemble all the statistic for batch analysis
for iresults in results_pool:
    fname, stats, extremes, dels, damage =  iresults
    mycruncher_lean_mc.add_output_stats(fname, stats, extremes, dels, damage)

In [12]:
# Results are the same as the full-memory approach above
mycruncher_lean.summary_stats["RootMc1"]['min']

DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: min, dtype: float64

In [13]:
mycruncher_lean_mc.dels

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,2557.248362,5801.906159,2074.270412
DLC2.3_2.out,2759.655817,4632.573610,2138.262799
DLC2.3_3.out,2791.460474,5839.945621,2213.542813


## Integrating outputs with a probability weighting (AEP, Damage, etc)

When running design load cases, not all windspeeds, or other environmental condition, occur with equal likelihood.  pCrunch provides a way to assign a probability to each output.  This probability can then weight a summation to compute annual energy production (AEP), or sum all Palmgren-Miner damages together.  Using a subset of the outputs is also a provided capability.

pCrunch provides a couple different ways to set the probabilities, either:
- Inflow wind speed using a Weibull or Rayleigh distribution for the site
- IEC turbine class with different average wind speeds that define a Weibull distribution
- Users can set the probability values directly.

In [14]:
# Set probability based on wind speed channel name, Weibull distribution average of 7.5 m/s (shape factor input optional)
mycruncher.set_probability_distribution('WindVxi', 7.5, kind='weibull', weibull_k=2.0)

# Or Rayleigh distribution using the same distribution average of 7.5 m/s
mycruncher.set_probability_distribution('WindVxi', 7.5, kind='rayleigh')

# If you only want to use some of the outputs, but not all of them
mycruncher.set_probability_distribution('WindVxi', 7.5, kind='weibull', idx=[0,2])

# If you would rather specify the inflow wind speed directly to use in the probability distribution
mycruncher.set_probability_distribution([8,10,12], 7.5, kind='weibull')

# Can also set the probability based on IEC turbine class, again using a channel name of user input of wind speeds
mycruncher.set_probability_turbine_class('WindVxi', 2)
mycruncher.set_probability_turbine_class([8,10,12], 2)

# A savvy user can set the probability values directly (they will be rescaled to sum to one no matter what)
mycruncher.prob = np.array([0.1, 0.5, 0.4])

Once the probabilities are set, the user can use them to calculate AEP or total fatigue accumulation across the scenarios represented by each output.  For the AEP calculation, the user must specify the channel name.  Additional loss factors or restriction to certain indices are optional inputs.

In [15]:
# Probability weighted and unweighted AEP values are returned
mycruncher.compute_aep('GenPwr')

(39945405528.0, 39675356190.000015)

In [16]:
# Or with loss factors and restricted by select outputs
mycruncher.compute_aep('GenPwr', loss_factor=0.15, idx=[0,2])

(33707934777.6, 33486451832.250023)

In [17]:
# Damage calculation does not require a channel name, as it uses the previously computed case-specific and channel-specific values.
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue()

In [18]:
dels_tot

,RootMc1,RootMc2,RootMc3
Weighted,2702.788217,5424.808463,2142.025341
Unweighted,2702.788217,5424.808463,2142.025341


In [19]:
dams_tot

,RootMc1,RootMc2,RootMc3
Weighted,0.0,0.0,0.0
Unweighted,0.0,0.0,0.0


In [20]:
# Select indices are also available to restrict the summation
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue(idx=[0,2])

## Other Batch Procressing

The Crunch class provides batch extensions of nearly all of the operations offered in the AeroelasticOutputs class.  This includes the add channel or drop channel utilities and all statistical functions.  For the statistics, unlike the AeroelasticOutput class, these batch versions are functions, not data properties.  The result is returned as a list, with each index corresponding to the output list.  Many of these statistics also vary by channel, so there are likely to be nested lists.  Also, some are unavailable in "lean / streaming" mode.

In [21]:
# Adding channel
mycruncher.calculate_channel('LSSGagMya + LSSGagMza', 'Test')

# Adding Load Roses
lr = {'TwrBs': ['TwrBsFxt', 'TwrBsFyt']}
mycruncher.add_load_rose(lr, nsec=6)

# Dropping channels by string wildcard
mycruncher.drop_channel('Fair*')
mycruncher.drop_channel('Anch*')
mycruncher.drop_channel('Spn*')
mycruncher.drop_channel('Root*')
mycruncher.drop_channel('Wave*')
mycruncher.drop_channel('Ptfm*')
mycruncher.drop_channel('Tw*')
mycruncher.drop_channel('Yaw*')

Added channel, TwrBs0
Added channel, TwrBs60
Added channel, TwrBs120
Added channel, TwrBs180
Added channel, TwrBs240
Added channel, TwrBs300
Added channel, TwrBs0
Added channel, TwrBs60
Added channel, TwrBs120
Added channel, TwrBs180
Added channel, TwrBs240
Added channel, TwrBs300
Added channel, TwrBs0
Added channel, TwrBs60
Added channel, TwrBs120
Added channel, TwrBs180
Added channel, TwrBs240
Added channel, TwrBs300


In [22]:
# Indices to the minimum value for each channel
mycruncher.idxmins()

array([[  0, 448,   0,   0, 401, 401,   0,   0,   0,   0, 258, 798, 798,
        490, 262, 505, 284, 486, 247, 473, 788, 653, 552, 209, 553, 222,
          0, 522, 286, 137, 406, 507, 489, 798, 453, 555, 406, 492],
       [  0, 450,   0,   0, 401, 401,   0,   0,   0,   0, 373, 798, 798,
        490, 224, 507, 412, 487, 249, 474, 789, 656, 617, 720, 785,  18,
          0, 522, 288, 260, 406, 508, 489, 798, 453, 555, 406, 493],
       [  0, 450,   0,   0, 401, 401,   0,   0,   0,   0, 260, 798, 798,
        490, 431, 505, 412, 486, 249, 474, 788, 654, 617, 721, 553,  19,
          0, 522, 288, 260, 406, 506, 488, 798, 453, 555, 406, 492]])

In [23]:
# Indices to the maximum value for each channel
mycruncher.idxmaxs()

array([[800, 502,   0,   0, 313, 315,   0, 630, 630, 630, 487, 436, 438,
        364, 677, 257, 465, 234, 563, 284, 512, 555, 403, 298, 655, 551,
          0, 232, 109, 314, 289, 567, 515, 448, 292, 231, 289, 555],
       [800, 505,   0,   0, 307, 313,   0, 630, 630, 630, 260, 438, 438,
        363, 683, 259, 466, 235, 565, 285, 511, 783, 403, 552, 657, 618,
          0, 407, 114, 316, 288, 561, 515, 449, 291, 407, 289, 554],
       [800, 505,   0,   0, 307, 311,   0, 630, 630, 630, 486, 438, 438,
        600, 686, 262, 466, 238, 564, 288, 511, 782, 403, 552, 655, 617,
          0, 407, 113, 316, 288, 590, 515, 449, 291, 407, 289, 553]])

In [24]:
# Minimum value of each channel
mycruncher.minima()

array([[ 4.00000000e+01,  8.20000000e+00,  0.00000000e+00,
         0.00000000e+00, -8.12000000e-01, -5.42000000e-01,
         0.00000000e+00, -7.48000000e-01, -7.23000000e-01,
         0.00000000e+00, -8.32000000e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.56000000e-02,
        -3.20000000e-02, -3.14000000e+00, -2.13000000e-02,
        -3.07000000e-01, -2.09000000e+00, -9.38000000e-01,
         0.00000000e+00, -3.08000000e+00, -8.94000000e-01,
         0.00000000e+00, -3.44000000e+00, -1.22000000e+00,
         0.00000000e+00,  9.61000000e+00,  9.39000000e+00,
         9.52000000e+00, -2.01000000e+00, -1.32000000e-01,
        -4.79000000e-01, -8.20000000e-01, -4.85000000e-02,
         0.00000000e+00,  1.95000000e+01, -1.29000000e-01,
        -1.01000000e+00, -1.07000000e-01, -5.44000000e+00,
        -1.29000000e+00, -3.51000000e-01, -1.92000000e-02,
        -3.52000000e-01, -1.27000000e+02, -2.12000000e+0

In [25]:
# Maximum value of each channel
mycruncher.maxima()

array([[ 8.00000000e+01,  1.27000000e+01,  0.00000000e+00,
         0.00000000e+00,  1.11000000e+00,  8.75000000e-01,
         0.00000000e+00,  6.70000000e-01,  6.80000000e-01,
         0.00000000e+00,  5.37000000e-01,  2.79000000e+03,
         2.73000000e+01,  0.00000000e+00,  9.00000000e+01,
         9.00000000e+01,  9.00000000e+01,  3.60000000e+02,
         1.11000000e+01,  1.09000000e+03,  6.38000000e-03,
         1.29000000e+00,  4.91000000e+00,  1.23000000e+00,
         0.00000000e+00,  4.58000000e+00,  1.57000000e+00,
         0.00000000e+00,  4.72000000e+00,  2.10000000e+00,
         0.00000000e+00,  6.55000000e+01,  6.54000000e+01,
         6.54000000e+01,  2.19000000e+00,  1.76000000e-01,
         5.19000000e-01,  6.94000000e-01,  2.72000000e-02,
         0.00000000e+00,  3.72000000e+01,  3.93000000e-02,
         6.39000000e-01,  1.85000000e-01,  5.02000000e+00,
         3.11000000e-01,  2.67000000e-01,  1.03000000e-02,
         3.83000000e-01,  2.50000000e+02,  1.70000000e+0

In [26]:
# Maximum value of absolute values of each channel
mycruncher.absmaxima()

array([[8.000e+01, 1.270e+01, 0.000e+00, 0.000e+00, 2.790e+03, 2.730e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.600e+02, 1.110e+01,
        1.090e+03, 2.130e-02, 1.290e+00, 4.910e+00, 1.230e+00, 4.580e+00,
        1.570e+00, 4.720e+00, 2.100e+00, 2.190e+00, 1.760e-01, 5.190e-01,
        8.200e-01, 4.850e-02, 0.000e+00, 7.590e+02, 1.140e+03, 1.130e+03,
        2.650e+03, 3.840e+03, 3.590e+03, 8.830e+00, 4.660e-01, 1.130e+00,
        6.240e-02, 4.930e+03],
       [8.000e+01, 1.272e+01, 0.000e+00, 0.000e+00, 2.866e+03, 2.780e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.595e+02, 1.128e+01,
        1.103e+03, 2.337e-02, 1.418e+00, 4.852e+00, 1.152e+00, 4.701e+00,
        1.658e+00, 4.526e+00, 2.183e+00, 2.472e+00, 1.646e-01, 5.903e-01,
        9.164e-01, 5.223e-02, 0.000e+00, 7.869e+02, 1.128e+03, 1.117e+03,
        2.701e+03, 3.969e+03, 3.930e+03, 8.988e+00, 4.801e-01, 1.171e+00,
        6.365e-02, 5.283e+03],
       [8.000e+01, 1.272e+01, 0.000e+00, 0.000e+00

In [27]:
# The range of data values (max - min)
mycruncher.ranges()

array([[4.00000000e+01, 4.50000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.92200000e+00, 1.41700000e+00, 0.00000000e+00, 1.41800000e+00,
        1.40300000e+00, 0.00000000e+00, 1.36900000e+00, 2.79000000e+03,
        2.73000000e+01, 0.00000000e+00, 9.00000000e+01, 9.00000000e+01,
        9.00000000e+01, 3.59924400e+02, 1.11320000e+01, 1.09314000e+03,
        2.76800000e-02, 1.59700000e+00, 7.00000000e+00, 2.16800000e+00,
        0.00000000e+00, 7.66000000e+00, 2.46400000e+00, 0.00000000e+00,
        8.16000000e+00, 3.32000000e+00, 0.00000000e+00, 5.58900000e+01,
        5.60100000e+01, 5.58800000e+01, 4.20000000e+00, 3.08000000e-01,
        9.98000000e-01, 1.51400000e+00, 7.57000000e-02, 0.00000000e+00,
        1.77000000e+01, 1.68300000e-01, 1.64900000e+00, 2.92000000e-01,
        1.04600000e+01, 1.60100000e+00, 6.18000000e-01, 2.95000000e-02,
        7.35000000e-01, 3.77000000e+02, 3.82000000e+02, 5.62000000e+02,
        8.47000000e+03, 1.40400000e+04, 1.38200000e+02, 3.780000

In [28]:
# Channel indices which vary in time
mycruncher.variable()

array([[ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37],
       [ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37],
       [ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37]])

In [29]:
# Channel indices which are constant in time
mycruncher.constant()

array([[ 2,  3,  6, 26],
       [ 2,  3,  6, 26],
       [ 2,  3,  6, 26]])

In [30]:
# Sum of channel values over time
mycruncher.sums()

array([[ 4.80600000e+04,  7.58900000e+03,  0.00000000e+00,
         0.00000000e+00,  9.89530000e+05,  1.00830000e+04,
         0.00000000e+00,  2.55375000e+04,  2.55375000e+04,
         2.55375000e+04,  1.44128285e+05,  5.74874039e+03,
         5.57390046e+05, -4.14189357e-01,  2.00572446e+02,
         1.55066591e+03, -3.54332400e+01,  1.52463849e+03,
         9.82605120e+01,  1.48320427e+03, -2.38651510e+02,
        -1.95322500e+01, -1.30349040e+00, -4.35471540e+01,
         1.17890310e+02, -1.10558919e+01,  0.00000000e+00,
         2.62518510e+05,  1.95963330e+04,  1.82839544e+05,
         8.71292238e+05,  4.83505880e+05, -5.12910800e+04,
         4.01501454e+03,  1.58754217e+02,  3.98490080e+02,
         2.08073147e+01,  4.32214800e+05],
       [ 4.80600000e+04,  7.58870100e+03,  0.00000000e+00,
         0.00000000e+00,  1.02185900e+06,  1.03015900e+04,
         0.00000000e+00,  2.55375000e+04,  2.55375000e+04,
         2.55375000e+04,  1.46322747e+05,  5.81471523e+03,
         5.64

In [31]:
# Sum of channel values over time to the second power
mycruncher.sums_squared()

array([[2.99066700e+06, 7.23318632e+04, 0.00000000e+00, 0.00000000e+00,
        2.45104090e+09, 2.53956520e+05, 0.00000000e+00, 1.99462425e+06,
        1.99462425e+06, 1.99462425e+06, 3.26158758e+07, 5.51322865e+04,
        5.18293747e+08, 1.52310134e-02, 3.39176067e+02, 6.81489156e+03,
        2.66962590e+02, 6.77778710e+03, 3.74944460e+02, 6.92880700e+03,
        5.28852071e+02, 8.93030853e+02, 2.49956593e+00, 5.45906269e+01,
        1.66439403e+02, 4.62125266e-01, 0.00000000e+00, 1.84694760e+08,
        4.01064942e+08, 5.08073920e+08, 2.49756762e+09, 2.46152297e+09,
        1.85959911e+09, 2.71241855e+04, 7.06899358e+01, 4.08180563e+02,
        1.37977617e+00, 3.49188164e+09],
       [2.99066700e+06, 7.23262441e+04, 0.00000000e+00, 0.00000000e+00,
        2.61382196e+09, 2.65084568e+05, 0.00000000e+00, 1.99462425e+06,
        1.99462425e+06, 1.99462425e+06, 3.34249703e+07, 5.64178526e+04,
        5.30965262e+08, 1.81228244e-02, 4.09763475e+02, 7.04301256e+03,
        2.74098853e+02,

In [32]:
# Sum of channel values over time to the third power
mycruncher.sums_cubed()

array([[ 1.92288060e+08,  6.94298326e+05,  0.00000000e+00,
         0.00000000e+00,  6.09512315e+12,  6.40731260e+06,
         0.00000000e+00,  1.65847398e+08,  1.65847398e+08,
         1.65847398e+08,  8.23177046e+09,  5.53178222e+05,
         5.04221025e+11, -1.55543032e-04,  3.65344809e+02,
         2.58163535e+04,  1.20947939e+01,  2.46568356e+04,
         2.05625057e+02,  2.45975817e+04,  7.10796755e+01,
         9.96182890e+01, -1.48520740e-02, -4.35589968e+00,
         1.53278176e+01, -1.34159296e-02,  0.00000000e+00,
         1.01753538e+11, -3.22542353e+10,  1.49847668e+11,
         5.84157948e+12,  1.43604884e+12,  5.65690138e+11,
         1.93195077e+05,  2.86981674e+01,  3.38936129e+02,
         7.67175294e-02,  3.45922125e+12],
       [ 1.92288060e+08,  6.94219888e+05,  0.00000000e+00,
         0.00000000e+00,  6.71230136e+12,  6.83293668e+06,
         0.00000000e+00,  1.65847398e+08,  1.65847398e+08,
         1.65847398e+08,  8.47749243e+09,  5.72455267e+05,
         5.22

In [33]:
# Sum of channel values over time to the fourth power
mycruncher.sums_fourth()

array([[1.27193675e+10, 6.72058716e+06, 0.00000000e+00, 0.00000000e+00,
        1.52191817e+16, 1.61942556e+08, 0.00000000e+00, 1.41881502e+10,
        1.41881502e+10, 1.41881502e+10, 2.23468362e+12, 5.63005878e+06,
        4.97577410e+14, 2.75297219e-06, 4.38130767e+02, 1.06767943e+05,
        1.74151600e+02, 1.01872083e+05, 4.17986394e+02, 1.04873960e+05,
        9.00199729e+02, 2.22800798e+03, 2.07158246e-02, 7.26447975e+00,
        6.04853833e+01, 5.39994955e-04, 0.00000000e+00, 6.62801332e+13,
        3.18222414e+14, 4.30309506e+14, 1.45026648e+16, 1.32816255e+16,
        1.00332283e+16, 1.40368241e+06, 1.20486157e+01, 3.26802571e+02,
        4.46744190e-03, 3.85369085e+16],
       [1.27193675e+10, 6.71962967e+06, 0.00000000e+00, 0.00000000e+00,
        1.73077680e+16, 1.76438826e+08, 0.00000000e+00, 1.41881502e+10,
        1.41881502e+10, 1.41881502e+10, 2.30428942e+12, 5.89126197e+06,
        5.21876656e+14, 4.09519763e-06, 6.40737567e+02, 1.13515183e+05,
        1.71822081e+02,

In [34]:
# Second moment of the timeseries for each channel
mycruncher.second_moments()

array([[1.33666667e+02, 5.37563724e-01, 0.00000000e+00, 0.00000000e+00,
        1.53384134e+06, 1.58591217e+02, 0.00000000e+00, 1.47370425e+03,
        1.47370425e+03, 1.47370425e+03, 8.34218479e+03, 1.73206485e+01,
        1.62826473e+05, 1.87476156e-05, 3.60739343e-01, 4.76022226e+00,
        3.31329783e-01, 4.83865350e+00, 4.53046962e-01, 5.22143747e+00,
        5.71470377e-01, 1.11430033e+00, 3.11790852e-03, 6.51974319e-02,
        1.86127885e-01, 3.86423328e-04, 0.00000000e+00, 1.23167724e+05,
        5.00106768e+05, 5.82195026e+05, 1.93484970e+06, 2.70869585e+06,
        2.31749656e+06, 8.73772149e+00, 4.89708356e-02, 2.62091685e-01,
        1.04777949e-03, 4.06824110e+06],
       [1.33666667e+02, 5.37621765e-01, 0.00000000e+00, 0.00000000e+00,
        1.63571375e+06, 1.65538992e+02, 0.00000000e+00, 1.47370425e+03,
        1.47370425e+03, 1.47370425e+03, 8.35886306e+03, 1.77365476e+01,
        1.66963034e+05, 2.23382251e-05, 4.28063595e-01, 4.87968906e+00,
        3.41619997e-01,

In [35]:
# Third moment of the timeseries for each channel
mycruncher.third_moments()

array([[ 0.00000000e+00,  1.04578434e+00,  0.00000000e+00,
         0.00000000e+00,  3.94767725e+07,  1.54227646e+01,
         0.00000000e+00,  3.36895113e+04,  3.36895113e+04,
         3.36895113e+04, -5.20236650e+04, -5.19943844e+01,
        -4.73889823e+07, -1.64965164e-07,  1.69420113e-01,
        -2.67129717e+00,  5.91565473e-02, -3.74347926e+00,
         8.81354619e-02, -4.64587100e+00,  6.25981817e-01,
         2.05897979e-01, -3.31601828e-06,  5.35617821e-03,
        -6.62345831e-02,  1.88154176e-06,  0.00000000e+00,
        -2.92704786e+07, -7.69871917e+07, -2.23500517e+08,
        -3.08118295e+08, -3.33225423e+09,  1.15168683e+09,
        -1.61412350e+01, -1.07475623e-03, -9.11508028e-02,
        -3.40501759e-06, -2.42407639e+09],
       [ 0.00000000e+00,  1.04733005e+00,  0.00000000e+00,
         0.00000000e+00,  4.34894071e+07,  1.63269772e+01,
         0.00000000e+00,  3.36895113e+04,  3.36895113e+04,
         3.36895113e+04, -9.31343873e+04, -5.41397262e+01,
        -4.94

In [36]:
# Fourth moment of the timeseries for each channel
mycruncher.fourth_moments()

array([[3.21601332e+04, 3.44360404e+00, 0.00000000e+00, 0.00000000e+00,
        2.43097390e+12, 2.55095341e+04, 0.00000000e+00, 3.39569151e+06,
        3.39569151e+06, 3.39569151e+06, 1.58498572e+08, 5.15307667e+02,
        4.55459785e+10, 3.06556334e-09, 2.37642092e-01, 3.28922739e+01,
        2.23991186e-01, 3.73738647e+01, 4.37451084e-01, 4.61649792e+01,
        1.55761531e+00, 2.79764035e+00, 2.57913196e-05, 9.06909402e-03,
        8.98454452e-02, 3.00027025e-07, 0.00000000e+00, 3.02030150e+10,
        4.03019005e+11, 5.56559180e+11, 4.31031003e+12, 1.85725589e+13,
        1.27638339e+13, 1.27548618e+02, 2.80917505e-03, 1.38923674e-01,
        1.23362554e-06, 4.61511943e+13],
       [3.21601332e+04, 3.45256143e+00, 0.00000000e+00, 0.00000000e+00,
        2.76447638e+12, 2.77911904e+04, 0.00000000e+00, 3.39569151e+06,
        3.39569151e+06, 3.39569151e+06, 1.57628652e+08, 5.41849680e+02,
        4.80084271e+10, 4.57357607e-09, 3.25917050e-01, 3.38973659e+01,
        2.17601724e-01,

In [37]:
# Mean of channel values over time
mycruncher.means()

array([[ 6.00000000e+01,  9.47440699e+00,  0.00000000e+00,
         0.00000000e+00,  7.65198252e-03,  3.38654182e-03,
         0.00000000e+00,  2.86660924e-03,  1.07333071e-02,
         0.00000000e+00, -1.19349938e-03,  1.23536829e+03,
         1.25880150e+01,  0.00000000e+00,  3.18820225e+01,
         3.18820225e+01,  3.18820225e+01,  1.79935436e+02,
         7.17695430e+00,  6.95867723e+02, -5.17090333e-04,
         2.50402554e-01,  1.93591250e+00, -4.42362547e-02,
         0.00000000e+00,  1.90341885e+00,  1.22672300e-01,
         0.00000000e+00,  1.85169072e+00, -2.97941960e-01,
         0.00000000e+00,  4.78828340e+01,  5.53099001e+01,
         5.65999376e+01, -2.43848315e-02, -1.62732884e-03,
        -5.43659850e-02,  1.47178914e-01, -1.38026115e-02,
         0.00000000e+00,  3.25186017e+01, -4.99510314e-02,
        -1.34598809e-01,  4.10221463e-02,  4.49336192e-01,
        -2.49922965e-01, -6.27209320e-02, -4.76854682e-05,
         3.68551061e-03,  1.06168428e+02, -9.16578027e+0

In [38]:
# Median of channel values over time
mycruncher.medians()

array([[ 6.00000000e+01,  9.40000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.52000000e-02, -2.76000000e-02,
         0.00000000e+00, -1.29000000e-02, -1.86000000e-03,
         0.00000000e+00,  6.05000000e-02,  2.30000000e+03,
         2.40000000e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.92000000e+02,
         1.00000000e+01,  9.74000000e+02,  9.87000000e-06,
        -8.82000000e-02,  3.34000000e+00, -8.95000000e-02,
         0.00000000e+00,  3.40000000e+00,  5.27000000e-02,
         0.00000000e+00,  3.41000000e+00, -4.79000000e-01,
         0.00000000e+00,  5.38000000e+01,  6.38000000e+01,
         6.34000000e+01, -7.60000000e-02,  1.91000000e-03,
        -5.33000000e-02,  2.78000000e-01, -1.54000000e-02,
         0.00000000e+00,  3.41000000e+01, -3.82000000e-02,
        -9.87000000e-02,  5.17000000e-02,  1.07000000e+00,
         9.09000000e-02, -4.90000000e-02, -2.78000000e-05,
         1.08000000e-02,  1.57000000e+02,  1.12000000e+0

In [39]:
# Standard deviation of channel values over time
mycruncher.stddevs()

array([[1.15614301e+01, 7.33187373e-01, 0.00000000e+00, 0.00000000e+00,
        4.22167785e-01, 2.98854589e-01, 0.00000000e+00, 2.96800509e-01,
        2.80448935e-01, 0.00000000e+00, 2.83252233e-01, 1.23848349e+03,
        1.25933005e+01, 0.00000000e+00, 3.83888558e+01, 3.83888558e+01,
        3.83888558e+01, 9.13355615e+01, 4.16180832e+00, 4.03517624e+02,
        4.32985169e-03, 6.00615803e-01, 2.18179336e+00, 5.75612528e-01,
        0.00000000e+00, 2.19969396e+00, 6.73087633e-01, 0.00000000e+00,
        2.28504649e+00, 7.55956598e-01, 0.00000000e+00, 1.76632092e+01,
        1.57823044e+01, 1.45667500e+01, 1.05560425e+00, 5.58382353e-02,
        2.55337878e-01, 4.31425410e-01, 1.96576532e-02, 0.00000000e+00,
        5.00503028e+00, 5.03460156e-02, 3.75609084e-01, 7.12951653e-02,
        2.70958397e+00, 6.01353472e-01, 1.51377511e-01, 4.02569647e-03,
        1.55848509e-01, 1.04680766e+02, 1.17135641e+02, 1.70983981e+02,
        2.38836378e+03, 3.92927918e+03, 2.83745885e+01, 1.103552

In [40]:
# Skew of channel values over time
mycruncher.skews()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:416: RuntimeWarning: invalid value encountered in divide
  return self.third_moments / np.sqrt(self.second_moments) ** 3


array([[ 0.        ,  2.65336626,         nan,         nan,  0.02078122,
         0.00772224,         nan,  0.59549649,  0.59549649,  0.59549649,
        -0.06827807, -0.72129051, -0.7212567 , -2.03223151,  0.78194229,
        -0.25720616,  0.3101788 , -0.35171318,  0.28902529, -0.38938752,
         1.44900917,  0.17504459, -0.01904679,  0.32174313, -0.82483607,
         0.24769592,         nan, -0.67714933, -0.21768293, -0.50312488,
        -0.11448448, -0.74747688,  0.32644179, -0.62494164, -0.09917534,
        -0.67932975, -0.10039556, -0.29541754],
       [ 0.        ,  2.65685773,         nan,         nan,  0.02078847,
         0.00766576,         nan,  0.59549649,  0.59549649,  0.59549649,
        -0.12186788, -0.72479038, -0.72437278, -2.21204139,  0.78021022,
        -0.25864842,  0.22294603, -0.37447278,  0.44241491, -0.41560236,
         1.54672098,  0.17733024, -0.10541872,  0.23070655, -0.90416767,
         0.29340951,         nan, -0.7905145 , -0.26053364, -0.5054796 ,
   

In [41]:
# Kurtosis of channel values over time
mycruncher.kurtosis()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:420: RuntimeWarning: invalid value encountered in divide
  return self.fourth_moments / self.second_moments ** 2


array([[ 1.79999626, 11.91662575,         nan,         nan,  1.03328332,
         1.01424824,         nan,  1.56353482,  1.56353482,  1.56353482,
         2.27753858,  1.71766437,  1.71790841,  8.72204274,  1.82614957,
         1.45157486,  2.04037492,  1.59631635,  2.13129248,  1.69329444,
         4.76949898,  2.25313579,  2.65305849,  2.13355049,  2.59342195,
         2.00924873,         nan,  1.99093218,  1.61138777,  1.6420046 ,
         1.15136769,  2.53134453,  2.37652806,  1.67062648,  1.1713961 ,
         2.02241265,  1.12368238,  2.78849312],
       [ 1.79999626, 11.94504332,         nan,         nan,  1.03323302,
         1.01416014,         nan,  1.56353482,  1.56353482,  1.56353482,
         2.25600854,  1.72242627,  1.72217318,  9.16555105,  1.77864998,
         1.42357926,  1.86455742,  1.64399072,  2.34040176,  1.70666827,
         5.06379094,  2.73034727,  2.71665559,  2.09867975,  2.79245263,
         2.43625957,         nan,  2.08109812,  1.60028736,  1.67032893,
   

In [42]:
# Integration of channel values over time
mycruncher.integrated()

array([[ 2.40000000e+03,  3.78980000e+02,  0.00000000e+00,
         0.00000000e+00,  3.18586900e-01,  1.45506000e-01,
         0.00000000e+00,  1.23832700e-01,  4.33616450e-01,
         0.00000000e+00, -5.73996500e-02,  4.94185000e+04,
         5.03547500e+02,  0.00000000e+00,  1.27462500e+03,
         1.27462500e+03,  1.27462500e+03,  7.19341423e+03,
         2.87187817e+02,  2.78452803e+04, -2.07112429e-02,
         1.00038398e+01,  7.74383080e+01, -1.78914950e+00,
         0.00000000e+00,  7.61321998e+01,  4.92485060e+00,
         0.00000000e+00,  7.40774884e+01, -1.18943255e+01,
         0.00000000e+00,  1.91505500e+03,  2.21192900e+03,
         2.26502450e+03, -1.01791250e+00, -6.63082700e-02,
        -2.18417020e+00,  5.90296550e+00, -5.51826343e-01,
         0.00000000e+00,  1.30115750e+03, -1.99829631e+00,
        -5.37945730e+00,  1.64098696e+00,  1.80961645e+01,
        -9.98462475e+00, -2.50321332e+00, -1.74655300e-03,
         1.45464700e-01,  4.24779055e+03, -3.73289500e+0

In [43]:
# Special instance of the integration that specifically uses
# the Power channel string to integrate over time and calculate energy
mycruncher.compute_energy('GenPwr')

DLC2.3_1.out    49418.500
DLC2.3_2.out    51033.000
DLC2.3_3.out    50520.175
Name: integrated, dtype: float64

In [44]:
# Total travel across simulation- useful for pitch drives and yaw drivers
mycruncher.total_travel('BldPitch1')

array([90., 90., 90.])